# Singularityのアンロード

---

構築したOpenHPC環境で[Singularity](https://sylabs.io/singularity/)がデフォルトでロードされないように設定します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境の各ノードに対してAnsibleで操作できるように設定されている

各ノードに対してAnsibleによる操作が行えることを確認します。操作対象となる UnitGroup 名を指定してください。

In [ ]:
ugroup_name = 'OpenHPC'

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping

## 設定変更

Singularityをデフォルトではロードされないようにするために `/etc/profile.d/` の設定ファイルを変更します。

[Lmod](https://lmod.readthedocs.io/en/latest/index.html)がデフォルトでロードするモジュールの一覧を表示してみます。

In [ ]:
!ansible {ugroup_name} -m shell -a 'module list'

Lmod がデフォルトでロードするモジュールは `/etc/profile.d/` にある `lmod.sh`, `lmod.csh` で設定されています。Singularityをロードしないように設定ファイルを書き換えます。

In [ ]:
# 書き換え対象のファイル
files = ['/etc/profile.d/lmod.sh', '/etc/profile.d/lmod.csh']

for file in files:
    !ansible {ugroup_name} -m lineinfile -b -a \
        'path={file} backup=yes regexp="module\s+try-add\s+singularity" state=absent'

設定が変更されたことを確認するために、ロードモジュールの内容を再度確認してみます。

In [ ]:
!ansible {ugroup_name} -m shell -a 'module list'

以下のように `singularity/3.4.1` がロードモジュールに表示されていなければ、設定変更が行われたことが確認できます。
```
Currently Loaded Modules:
  1) autotools
  2) prun/1.3
  3) gnu8/8.3.0
  4) openmpi3/3.1.4
  5) ohpc
```